In [ ]:
import cv2
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from scipy import ndimage

import matplotlib.pyplot as plt

# Load frames from noise removal notebook
# Using memory-mapped loading + chunked processing to avoid large allocations

def _to_uint8(gray):
    """Per-frame robust scaling to uint8 for Canny input."""
    gray = np.nan_to_num(gray)
    # Use percentiles to avoid blow-up from outliers
    lo, hi = np.percentile(gray, [1, 99])
    if hi == lo:
        return np.zeros_like(gray, dtype=np.uint8)
    scaled = np.clip((gray - lo) / (hi - lo) * 255, 0, 255)
    return scaled.astype(np.uint8)

def detect_crack_edges(frame, threshold=None):
    """Detect crack edges using edge detection."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) if len(frame.shape) == 3 else frame
    gray_u8 = _to_uint8(gray)
    if threshold is None:
        threshold = np.percentile(gray_u8, 95)
    edges = cv2.Canny(gray_u8, threshold * 0.5, threshold)
    return edges

def extract_crack_features(frame, edges):
    """Extract crack length, tip position, and opening displacement (placeholder)."""
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    features = {
        'crack_length': 0,
        'tip_position': None,
        'opening_displacement': 0,
        'contour': None
    }
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        features['contour'] = largest_contour
        features['crack_length'] = cv2.arcLength(largest_contour, False)
        # Note: this uses the contour centroid as a coarse tip proxy; replace with endpoint logic if needed
        M = cv2.moments(largest_contour)
        if M['m00'] != 0:
            features['tip_position'] = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
    return features

def track_crack_growth(frames_source, chunk_size=100, visualize=False):
    """Track crack growth across frame sequence using chunked processing to limit memory."""
    crack_data = {
        'frame_idx': [],
        'length': [],
        'tip_x': [],
        'tip_y': [],
        'opening_displacement': []
    }
    n_frames = frames_source.shape[0]
    for start in range(0, n_frames, chunk_size):
        end = min(start + chunk_size, n_frames)
        chunk = frames_source[start:end]
        for local_idx, frame in enumerate(chunk):
            idx = start + local_idx
            edges = detect_crack_edges(frame)
            features = extract_crack_features(frame, edges)
            crack_data['frame_idx'].append(idx)
            crack_data['length'].append(features['crack_length'])
            tip = features['tip_position']
            crack_data['tip_x'].append(tip[0] if tip else None)
            crack_data['tip_y'].append(tip[1] if tip else None)
            crack_data['opening_displacement'].append(features['opening_displacement'])
    return crack_data

frames_path = Path('1508 20250613 105 kx Ceta Camera.npy')

# Process your frames with memory mapping and chunks to avoid OOM
if frames_path.exists():
    frames = np.load(frames_path, mmap_mode='r')
    chunk_size = 100
    crack_growth_data = track_crack_growth(frames, chunk_size=chunk_size)
    print(f"Tracked {len(crack_growth_data['frame_idx'])} frames (chunk_size={chunk_size})")
else:
    print("Frames file not found. Load or export frames from the noise removal notebook first.")

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\canny.cpp:768: error: (-215:Assertion failed) _src.depth() == CV_8U in function 'cv::Canny'
